In [19]:
import pandas as pd
import geopandas as gpd
import numpy as np
import requests

# 対象地域情報の読み込み

In [2]:
gdf_jcode = gpd.read_file('data/setup_2000_2020/jcode.geojson')
df_jcode_2000 = pd.read_csv('data/2000/jcode_2000.csv', dtype={'jcode_from': str, 'jcode_to': str})
list_jcode = df_jcode_2000['jcode_from'].to_list()
list_jcode_to = list(set(df_jcode_2000['jcode_from'].to_list()))

In [ ]:
df_jcode_2000

# 地価情報の取得

In [4]:
# pd.read_excel('https://www.mlit.go.jp/totikensangyo/content/001733704.xls')
df_chika = pd.read_excel('sources/001733704.xls', sheet_name='価格推移表', header=0, skiprows=[1])
nayose = pd.read_csv('sources/chika_nayose.csv')

In [5]:
df_chika['Unnamed: 0'] = df_chika['Unnamed: 0'].astype(str)
df_chika['level'] = df_chika['Unnamed: 0'].str[0:1]
df_chika['jcode'] = df_chika['Unnamed: 0'].str[1:6]
df_chika['type'] = df_chika['Unnamed: 0'].str[-2:]
df_chika['name'] = df_chika['Unnamed: 1'].ffill()
df_chika['分類'] = df_chika['Unnamed: 2']
df_chika.loc[df_chika['jcode'] == "14150", "jcode"] = "14209" # 相模原市のjcodeを修正
df_chika = df_chika.drop(columns=['Unnamed: 0', 'Unnamed: 1','Unnamed: 2'])
df_chika = df_chika[[
    'level',   'jcode',   'type',    'name',    '分類',
    '昭和50年', '昭和51年', '昭和52年', '昭和53年', '昭和54年', '昭和55年', '昭和56年', '昭和57年',
    '昭和58年', '昭和59年', '昭和60年', '昭和61年', '昭和62年', '昭和63年', '平成元年', '平成2年',
    '平成3年',  '平成4年',  '平成5年',  '平成6年',  '平成7年',  '平成8年',  '平成9年',  '平成10年',
    '平成11年', '平成12年', '平成13年', '平成14年', '平成15年', '平成16年', '平成17年', '平成18年',
    '平成19年', '平成20年', '平成21年', '平成22年', '平成23年', '平成24年', '平成25年', '平成26年',
    '平成27年', '平成28年', '平成29年', '平成30年', '平成31年', '令和2年',  '令和3年',  '令和4年',
    '令和5年',  '令和6年'
]]

In [6]:
df_chika_res = df_chika[['jcode','分類','平成12年']].query('分類 == "住宅"').rename(columns={'平成12年':'res_price'})
df_chika_com = df_chika[['jcode','分類','平成12年']].query('分類 == "商業"').rename(columns={'平成12年':'com_price'})

df_chika_res = df_chika_res.query('jcode in @list_jcode')
df_chika_com = df_chika_com.query('jcode in @list_jcode')

In [ ]:
# 2020年居住地平均価格を参考に、300,000円/坪で正規化
print('居住地価格平均値:', np.mean(df_chika_res['res_price'].to_numpy()))
print('居住地価格中央値:', np.median(df_chika_res['res_price'].to_numpy()))

In [ ]:
df_chika_res['q_i'] = df_chika_res['res_price']/300000
df_chika_com['Q_j'] = df_chika_com['com_price']/300000
print(df_chika_res['q_i'].mean())
print(df_chika_com['Q_j'].mean())

In [9]:
df_q_i = df_chika_res[['jcode','q_i']].round(6).reset_index(drop=True)
df_Q_j = df_chika_com[['jcode','Q_j']].round(6).reset_index(drop=True)
df_Q_j.loc[df_Q_j['Q_j'].isnull(),'Q_j'] = df_q_i.loc[df_Q_j['Q_j'].isnull(),'q_i'] # Q_jがない場合はq_iで置き換える

In [ ]:
df_Q_j

# 所得w_jの取得

In [ ]:
# 所得w_jの取得
URL = 'http://api.e-stat.go.jp/rest/3.0/app/json/getStatsData'
p = {
    'appId':'6f812f8eb28dc679954f0eef1472fca422edc08d',
    'lang':'J',
    'statsDataId':'0000081782',
    # 'metaGetFlg':'N',
    'explanationGetFlg':'N',
    'annotationGetFlg':'N',
    'cdCat02':'001',
    # 'lvArea':'2-',
    'cdAreaFrom': '08000',
    'cdAreaTo': '14999',
}

res = requests.get(URL,p)
text = res.json()
print('項目名：',text['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']['TITLE']['$'])
print('ヒット件数：',text['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']['TOTAL_NUMBER'])
text['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

In [17]:
df = pd.DataFrame(text['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE'])

def parse_wage(x):
   if   x=='001': return 100
   elif x=='002': return 250
   elif x=='003': return 350
   elif x=='004': return 450
   elif x=='005': return 600
   elif x=='006': return 850
   elif x=='007': return 1250
   elif x=='008': return 2000
   else: return 0

def parse_num(x):
   if x=='-': return 0
   else: return int(x)

df['weight'] = df['@cat01'].apply(parse_wage)
df['num'] = df['$'].astype(int)
df['jcode'] = df['@area']


wm = lambda x: np.average(x, weights=df.loc[x.index, "num"]).round(2)
df_w_j = df.groupby(by='jcode').agg(w_j=("weight", wm)).loc[list_jcode_to].reset_index().sort_values('jcode')

In [ ]:
# 2020年賃金率を参考に、300万円/年で正規化
print('賃金率平均値:', np.mean(df_w_j['w_j'].to_numpy()).round(2))
print('賃金率中央値:', np.median(df_w_j['w_j'].to_numpy()).round(2))

In [ ]:
df_w_j['w_j'] = (df_w_j['w_j']/300).round(6)
df_w_j

# 通勤割合λ_ijの取得

In [ ]:
From = '08000'
To = '14999'

URL = 'http://api.e-stat.go.jp/rest/3.0/app/json/getStatsData'
p = {
     'appId':'6f812f8eb28dc679954f0eef1472fca422edc08d',
     'lang':'J',
     'statsDataId':'0000033333',
     # 'metaGetFlg':'N',
     'explanationGetFlg':'N',
     'annotationGetFlg':'N',
     # 'cdArea':','.join(list),
     # # 'cdCat02':','.join(list)
     # 'lvArea':'4-',
     'cdAreaFrom': From,
     'cdAreaTo': To,
     # 'lvCat02':'4-',
     'cdCat01':'000',
     'cdCat02':'T01',
     'cdCat03':'001',
     }

res = requests.get(URL,p)
text = res.json()
print('項目名：',text['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']['TITLE']['$'])
print('ヒット件数：',text['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']['TOTAL_NUMBER'])

df_1 = pd.DataFrame(text['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE'])
df_1 = df_1[['@area', '$']].rename(columns={'@area':'from', '$':'value'})
df_1['to'] = df_1['from']
df_1 = df_1[['to','from', 'value']]
text['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

In [ ]:
URL = 'http://api.e-stat.go.jp/rest/3.0/app/json/getStatsData'
p = {
     'appId':'6f812f8eb28dc679954f0eef1472fca422edc08d',
     'lang':'J',
     'statsDataId':'0000033343',
     'startPosition':'1',
     # 'metaGetFlg':'N',
     'explanationGetFlg':'N',
     'annotationGetFlg':'N',
     # 'cdArea':','.join(list),
     # # 'cdCat02':','.join(list)
     # 'lvArea':'4-',
     'cdAreaFrom': From,
     'cdAreaTo': To,
     # 'lvCat02':'4-',
     'cdCat01From':From,
     'cdCat01To':To,
     'cdCat02':'000',
     'cdCat03':'001'
     }

res = requests.get(URL,p)
text = res.json()
print('項目名：',text['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']['TITLE']['$'])
print('ヒット件数：',text['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']['TOTAL_NUMBER'])

df_2 = pd.DataFrame(text['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE'])
df_2 = df_2[['@cat01', '@area', '$']].rename(columns={'@cat01':'to', '@area':'from', '$':'value'})
df_2 = df_2[df_2['from'] != df_2['to']]
text['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

In [18]:
df = pd.concat([df_1, df_2])

df_commute_ij = pd.DataFrame(0,columns=list_jcode,index=list_jcode)
for i in df.itertuples():
    if (i[1] not in list_jcode)|(i[2] not in list_jcode): continue
    if i[3] in ['-',None]: 
        df_commute_ij.at[i[2],i[1]]=0
    else: 
        if int(i[3]) > 100: df_commute_ij.at[i[2],i[1]]=int(i[3])
        # df_commute_ij.at[i[2],i[1]]=int(i[3])

In [ ]:
df_commute_ij

# 世帯あたり子供の数n_ijの取得

In [ ]:
URL = 'http://api.e-stat.go.jp/rest/3.0/app/json/getStatsData'
p = {
     'appId':'6f812f8eb28dc679954f0eef1472fca422edc08d',
     'lang':'J',
     'statsDataId':'0000032965',
     # 'metaGetFlg':'N',
     'explanationGetFlg':'N',
     'annotationGetFlg':'N',
     'cdCat01':'00700',
     'cdCat02':'000',
     'cdCat03':','.join(['200','201','202']),
     'cdCat04':'000',
     # 'lvArea':'5-',
     'cdAreaFrom': From,
     'cdAreaTo': To,
}

res = requests.get(URL,p)
text = res.json()
print('項目名：',text['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']['TITLE']['$'])
print('ヒット件数：',text['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']['TOTAL_NUMBER'])
text['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

In [21]:
df = pd.DataFrame(text['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE'])
df_children_i = pd.DataFrame(0,index=list_jcode,columns=[0])
for i in df.itertuples():
    try: 
        if i[8] == '-': pass
        else: df_children_i.at[i[5],0] += int(i[8])
    except: pass

arr_children_ij = df_children_i.to_numpy() * df_commute_ij.to_numpy()/df_commute_ij.sum(axis=1).to_numpy().reshape(1,-1).T
df_children_ij = pd.DataFrame(arr_children_ij,index=list_jcode, columns=list_jcode)
df_children_ij = (df_children_ij).fillna(0)

In [ ]:
df_children_ij

# 交通費用の算出

In [21]:
gdf_gov = gpd.read_file('data/setup_2000_2020/gov_poi.geojson',dtypes={'jcode':str})

In [22]:
df_time = pd.read_csv('sources/e-2_H10.csv', header=3, encoding='CP932')
df_time.columns = ['origin','destination','発ゾーン','着ゾーン','鉄道','路線バス・都電','自動車','２輪車','自転車','徒歩','その他','合計']
df_time['発ゾーン'] = df_time['発ゾーン'].str.strip(':').str.translate(str.maketrans('０１２３４５６７８９', '0123456789'))
df_time['着ゾーン'] = df_time['着ゾーン'].str.strip(':').str.translate(str.maketrans('０１２３４５６７８９', '0123456789'))
ptcode = gdf_gov['H10'].tolist()
df_time = df_time[(df_time['発ゾーン'].isin(ptcode))&(df_time['着ゾーン'].isin(ptcode))]

In [23]:
df_time = df_time.merge(gdf_gov[['jcode','H10']], left_on='発ゾーン', right_on='H10', how='left').rename(columns={'jcode':'ori_jcode'}).drop(columns='H10')
df_time = df_time.merge(gdf_gov[['jcode','H10']], left_on='着ゾーン', right_on='H10', how='left').rename(columns={'jcode':'dst_jcode'}).drop(columns='H10')

df_time = df_time.sort_values(['ori_jcode', 'dst_jcode'])
df_time['鉄道'] = df_time['鉄道'].where(df_time['鉄道'] !=0.0, df_time[['路線バス・都電','自動車']].apply(lambda x: np.round(np.mean(x),2), axis=1))

In [24]:
df_time_ij = (df_time.pivot_table(values='鉄道', index='ori_jcode', columns='dst_jcode', fill_value=540)/(60*24)).round(6)

In [ ]:
df_time_ij

In [ ]:
df_time_ij_sim = df_time_ij
df_time_ij_sim.loc['13101':'13123',:] = df_time_ij_sim.loc['13101':'13123',:]*0.50
df_time_ij_sim.loc[:,'13101':'13123'] = df_time_ij_sim.loc[:,'13101':'13123']*0.50
df_time_ij_sim.loc['13101':'13123','13101':'13123'] = df_time_ij_sim.loc['13101':'13123','13101':'13123']/0.50
df_time_ij_sim

運賃の参考
初乗り:　130円 (第84条)
距離運賃: 16.8円/km (第77条)
https://www.desktoptetsu.com/ryoki/ryokikaitei_jre_2.htm

In [27]:
gdf_gov = gdf_gov.to_crs(epsg=6677)
list_gov = gdf_gov['jcode'].to_list()
distance_matrix = gdf_gov.geometry.apply(lambda x: gdf_gov.distance(x)).values
distance_matrix = np.where(distance_matrix/1000 > 130/16.8, distance_matrix/1000*16.8, 130)
distance_matrix = distance_matrix * (22*12) / 3000000
distance_matrix = np.round(distance_matrix, 5)
df_cost_ij = pd.DataFrame(distance_matrix,index=list_gov, columns=list_gov)

In [ ]:
df_cost_ij

# 商業地比率の算出



住宅件数と事業所数から算出
- 令和3年延べ面積比率（区別）
  - https://www.toshiseibi.metro.tokyo.lg.jp/seisaku/tochi_c/pdf/tochi_r3/tochi_r3_67.csv
- 事業所数
  - https://api.e-stat.go.jp/rest/3.0/app/getStatsData?cdCat01=A%2CB%2CC%2CD%2CE%2CF%2CG%2CH%2CI%2CJ%2CK%2CL%2CM%2CN%2CO%2CP%2CQ%2CR&cdTab=102-2021&cdArea=13101%2C13102%2C13103%2C13104%2C13105%2C13106%2C13107%2C13108%2C13109%2C13110%2C13111%2C13112%2C13113%2C13114%2C13115%2C13116%2C13117%2C13118%2C13119%2C13120%2C13121%2C13122%2C13123&appId=&lang=J&statsDataId=0004005684&metaGetFlg=Y&cntGetFlg=N&explanationGetFlg=Y&annotationGetFlg=Y&sectionHeaderFlg=1&replaceSpChars=0
- 
- 住宅数
  - https://api.e-stat.go.jp/rest/3.0/app/getStatsData?cdCat01=1%2C2%2C3%2C4&cdCat02=00&cdArea=13101%2C13102%2C13103%2C13104%2C13105%2C13106%2C13107%2C13108%2C13109%2C13110%2C13111%2C13112%2C13113%2C13114%2C13115%2C13116%2C13117%2C13118%2C13119%2C13120%2C13121%2C13122%2C13123&cdCat03=0&appId=&lang=J&statsDataId=0003355607&metaGetFlg=Y&cntGetFlg=N&explanationGetFlg=Y&annotationGetFlg=Y&sectionHeaderFlg=1&replaceSpChars=0
- 

In [29]:
df_weight = pd.read_excel('https://www.toukei.metro.tokyo.lg.jp/tnenkan/2001/01qytia0032.xls', header=13, skiprows=[14,15,16,40,41,42,43,44,45,75,76,77,78,79,80,81,82,83] ,usecols=[2,17,18,19,20,21,22,23,24,25,26,27,28,29,31,32])

- Ａ農業 : o
- Ｂ林業 : o
- Ｃ漁業 : o
- Ｄ鉱業 : なし
- Ｅ建設業 : なし
- Ｆ製造業 : lm
- Ｇ電気・ガス・熱供給・水道業 : d
- Ｈ運輸・通信業 : n
- Ｉ卸売・小売業，飲食店 : efghi
- Ｊ金融・保険業 : efghi
- Ｋ不動産業 : efghi
- Ｌサービス業 : efghi
- 一戸建て : j
- その他住宅 : k
- 国・地方公共団体 : abc

- abc
  - a官公庁施設
  - b教育文化施設
  - c厚生医療施設
- d
  - d供給処理施設
- efghi
  - e事務所建築物
  - f専用商業施設
  - g住商併用施設
  - h宿泊遊興施設
  - iスポーツ興業施設
- j
  - j独立住宅
- k
  - k集合住宅
- lm
  - l専用工場
  - m住居併用工場
- n
  - n倉庫・運輸関係施設
- o
  - o農林漁業施設

In [ ]:
df_weight.columns = ['市区町村','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o']
df_weight['市区町村'] = df_weight['市区町村'].str.replace('　','')
df_weight = df_weight.merge(df_jcode_2000.query('都道府県=="東京都"')[['市区町村','jcode_from']], on='市区町村', how='left')
df_weight = df_weight.query('jcode_from in @list_jcode').reset_index(drop=True)
df_weight.head()

In [31]:
df_weight_i= pd.DataFrame(
    0,
    index=list_jcode,
    columns=['abc','d','efghi','j','k','lm','n','o']
)

In [ ]:
import statsmodels.api as sm
URL = 'http://api.e-stat.go.jp/rest/3.0/app/json/getStatsData'
p = {'appId':'6f812f8eb28dc679954f0eef1472fca422edc08d',
     'lang':'J',
     'statsDataId':'0000041316',
     'explanationGetFlg':'N',
     'annotationGetFlg':'N',
     # 'cdCat01':'A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R',
     # 'cdTab':'102-2021',
     # 'cdArea':','.join(jcode),
     'cdCat02':'001',
     # 'cdCat03':'000',
     # 'lvArea':'5-',
     'cdAreaFrom': From,
     'cdAreaTo': To
     }
res = requests.get(URL,p)
text = res.json()
print('項目名：',text['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']['TITLE']['$'])
print('ヒット件数：',text['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']['TOTAL_NUMBER'])
text['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

In [33]:
df = pd.DataFrame(text['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE'])
for i in df.itertuples():
    try: 
        if i[6] == '-': continue
        else: 
            if   i[1]=='003': df_weight_i.at[i[3],'o'] += int(i[6]) #Ａ農業 : o
            elif i[1]=='004': df_weight_i.at[i[3],'o'] += int(i[6]) #Ｂ林業 : o
            elif i[1]=='005': df_weight_i.at[i[3],'o'] += int(i[6]) #Ｃ漁業 : o
            elif i[1]=='007': continue #Ｄ鉱業 : なし
            elif i[1]=='008': continue #Ｅ建設業 : なし
            elif i[1]=='009': df_weight_i.at[i[3],'lm'] += int(i[6]) #Ｆ製造業 : lm
            elif i[1]=='010': df_weight_i.at[i[3],'d'] += int(i[6]) #Ｇ電気・ガス・熱供給・水道業 : d
            elif i[1]=='011': df_weight_i.at[i[3],'n'] += int(i[6]) #Ｈ運輸・通信業 : n
            elif i[1]=='012': df_weight_i.at[i[3],'efghi'] += int(i[6]) #Ｉ卸売・小売業，飲食店 : efghi
            elif i[1]=='016': df_weight_i.at[i[3],'efghi'] += int(i[6]) #Ｊ金融・保険業 : efghi
            elif i[1]=='017': df_weight_i.at[i[3],'efghi'] += int(i[6]) #Ｋ不動産業 : efghi
            elif i[1]=='018': df_weight_i.at[i[3],'efghi'] += int(i[6]) #Ｌサービス業 : efghi
    except: continue

In [ ]:
URL = 'http://api.e-stat.go.jp/rest/3.0/app/json/getStatsData'
p = {'appId':'6f812f8eb28dc679954f0eef1472fca422edc08d',
     'lang':'J',
     'statsDataId':'0000081637',
     # 'metaGetFlg':'N',
     'explanationGetFlg':'N',
     'annotationGetFlg':'N',
     'cdCat01':'000',
     'cdCat02':','.join(['002','005','008','013']),
     # 'cdCat03':'0',
     # 'cdArea':','.join(jcode),
     # 'cdCat02':','.join(jcode)
     # 'lvArea':'5-',
     'cdAreaFrom': From,
     'cdAreaTo': To
     }
res = requests.get(URL,p)
text = res.json()
print('項目名：',text['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']['TITLE']['$'])
print('ヒット件数：',text['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']['TOTAL_NUMBER'])
text['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

In [35]:
df = pd.DataFrame(text['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE'])
for i in df.itertuples():
    try: 
        if i[6] == '-': pass
        else: 
            if i[2]=='002': df_weight_i.at[i[3],'j'] += int(i[6]) # 一戸建て : j
            else: df_weight_i.at[i[3],'k'] += int(i[6]) # その他住宅 : k
    except: pass

In [ ]:
URL = 'http://api.e-stat.go.jp/rest/3.0/app/json/getStatsData'
p = {'appId':'6f812f8eb28dc679954f0eef1472fca422edc08d',
     'lang':'J',
     'statsDataId':'0000040993',
     # 'metaGetFlg':'N',
     'explanationGetFlg':'N',
     'annotationGetFlg':'N',
     'cdCat01':'001',
     'cdCat02':'044',
     # 'cdCat03':'000',
     'cdAreaFrom': From,
     'cdAreaTo': To
     }
res = requests.get(URL,p)
text = res.json()
print('項目名：',text['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']['TITLE']['$'])
print('ヒット件数：',text['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']['TOTAL_NUMBER'])
text['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

In [37]:
df =  pd.DataFrame(text['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE'])
for i in df.itertuples():
    try: 
        if i[6] == '-': pass
        else: 
            df_weight_i.at[i[3],'abc'] += int(i[6])
    except: pass

In [ ]:
df_weight_i

In [39]:
df_weight_i['all'] = df_weight_i['abc'] + df_weight_i['d'] + df_weight_i['efghi'] + df_weight_i['j'] + df_weight_i['k'] + df_weight_i['lm'] + df_weight_i['n'] + df_weight_i['o']
df_weight_i['abc'] = df_weight_i['abc'] / df_weight_i['all']
df_weight_i['d'] = df_weight_i['d'] / df_weight_i['all']
df_weight_i['efghi'] = df_weight_i['efghi'] / df_weight_i['all']
df_weight_i['j'] = df_weight_i['j'] / df_weight_i['all']
df_weight_i['k'] = df_weight_i['k'] / df_weight_i['all']
df_weight_i['lm'] = df_weight_i['lm'] / df_weight_i['all']
df_weight_i['n'] = df_weight_i['n'] / df_weight_i['all']
df_weight_i['o'] = df_weight_i['o'] / df_weight_i['all']

In [40]:
tokyo = df_weight['jcode_from'].to_list()
df_weight_tokyo = df_weight_i.reset_index(names='jcode').query('jcode in @tokyo').reset_index()
df_weight_others = df_weight_i.reset_index(names='jcode').query('jcode not in @tokyo').reset_index()

In [41]:
abc_res   = sm.OLS((df_weight['a']+df_weight['b']+df_weight['c']), df_weight_tokyo['abc']).fit()
d_res     = sm.OLS((df_weight['d']), df_weight_tokyo['d']).fit()
efghi_res = sm.OLS((df_weight['e']+df_weight['f']+df_weight['g']+df_weight['h']+df_weight['i']), df_weight_tokyo['efghi']).fit()
j_res     = sm.OLS((df_weight['j']), df_weight_tokyo['j']).fit()
k_res     = sm.OLS((df_weight['k']), df_weight_tokyo['k']).fit()
lm_res    = sm.OLS((df_weight['l']+df_weight['m']), df_weight_tokyo['lm']).fit()
n_res     = sm.OLS((df_weight['n']), df_weight_tokyo['n']).fit()
o_res     = sm.OLS((df_weight['o']), df_weight_tokyo['o']).fit()

In [42]:
df_floorparam = pd.DataFrame(
    [
        abc_res.params['abc'],
        d_res.params['d'],
        efghi_res.params['efghi'],
        j_res.params['j'],
        k_res.params['k'],
        lm_res.params['lm'],
        n_res.params['n'],
        o_res.params['o']
    ],
    columns = ['param'],
    index = ['abc','d','efghi','j','k','lm','n','o'],
)
params = df_floorparam.to_dict()['param']
df_floorparam.to_csv('sources/floorspace.csv')

In [43]:
df_weight_tokyo['com_ratio'] = df_weight[['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'l', 'm','n', 'o']].sum(axis=1)
df_weight_tokyo['res_ratio'] = df_weight[['j', 'k']].sum(axis=1)
df_weight_tokyo['theta_i']   = df_weight_tokyo['com_ratio']/(df_weight_tokyo['com_ratio']+df_weight_tokyo['res_ratio'] )

df_weight_others['com_ratio'] = (df_weight_others[['abc','d','efghi','lm','n','o']]*[params['abc'], params['d'], params['efghi'], params['lm'], params['n'], params['o']]).sum(axis=1)
df_weight_others['res_ratio'] = (df_weight_others[['j','k']]*[params['j'], params['k']]).sum(axis=1)
df_weight_others['theta_i']   =  df_weight_others['com_ratio']/(df_weight_others['com_ratio']+df_weight_others['res_ratio'])

df_theta = pd.concat([df_weight_others,df_weight_tokyo]).sort_values('jcode').reset_index()
df_theta_i = df_theta[['jcode','theta_i']].round(6)

In [ ]:
df_theta_i

# その他の指標

In [45]:
df_p_i = pd.DataFrame({
    'jcode': list_jcode,
    'p_i':1
})

In [46]:
df_param = pd.DataFrame({
    'alpha': [0.8],
    'gamma': [0.375],
    'psi': [0.25],
    'beta_cns': [0.6],
    'beta_flr': [0.2], 
    'beta_chd': [0.2]
})

In [23]:
df_scaler = pd.DataFrame({
    'T': [1],
    'L': [0.66],
    'N': [np.sum(df_commute_ij.to_numpy())]
})

# 対象年度のマトリックスから、対象地域コードに置き換えてAggregateする

In [24]:
def agg_sum_df(df):
    df_ij = df.merge(df_jcode_2000[['jcode_from', 'jcode_to']], left_index=True, right_on='jcode_from').drop(columns=['jcode_from']).set_index('jcode_to')
    df_ij = df_ij.T.merge(df_jcode_2000[['jcode_from', 'jcode_to']], left_index=True, right_on='jcode_from').drop(columns=['jcode_from']).set_index('jcode_to').T
    df_ij = df_ij.groupby(df_ij.index).sum().T.groupby(df_ij.index).sum().T
    return df_ij

def agg_mean_df(df):
    df_ij = df.merge(df_jcode_2000[['jcode_from', 'jcode_to']], left_index=True, right_on='jcode_from').drop(columns=['jcode_from']).set_index('jcode_to')
    df_ij = df_ij.T.merge(df_jcode_2000[['jcode_from', 'jcode_to']], left_index=True, right_on='jcode_from').drop(columns=['jcode_from']).set_index('jcode_to').T
    df_ij = df_ij.groupby(df_ij.index).mean().T.groupby(df_ij.index).mean().T
    return df_ij

In [25]:
df_commute_ij = agg_sum_df(df_commute_ij)
df_children_ij = agg_sum_df(df_children_ij)
df_n_ij = (df_children_ij/df_commute_ij).fillna(0).round(6)
df_Pi_ij = (df_commute_ij/np.sum(df_commute_ij.to_numpy())).round(9)

In [ ]:
df_Pi_ij

# データの保存

In [51]:
df_merged = df_jcode_2000.merge(df_w_j, left_on='jcode_from', right_on='jcode', how='left').drop(columns=['jcode'])
df_merged = df_merged.merge(df_q_i, left_on='jcode_from', right_on='jcode', how='left').drop(columns=['jcode'])
df_merged = df_merged.merge(df_Q_j, left_on='jcode_from', right_on='jcode', how='left').drop(columns=['jcode'])
df_merged = df_merged.merge(df_p_i, left_on='jcode_from', right_on='jcode', how='left').drop(columns=['jcode'])
df_merged = df_merged.merge(df_theta_i, left_on='jcode_from', right_on='jcode', how='left').drop(columns=['jcode'])
df_merged = df_merged.groupby(by='jcode_to').agg({'w_j':'mean','q_i':'mean','Q_j':'mean','p_i':'mean','theta_i':'mean'}).reset_index(names='jcode')

In [27]:
df_scaler.to_csv('data/2000/scaler.csv', index=False)
df_Pi_ij.to_csv('data/2000/Pi_ij.csv')
df_n_ij.to_csv('data/2000/n_ij.csv')

In [52]:
df_scaler.to_csv('data/2000/scaler.csv', index=False)
df_param.to_csv('data/2000/param.csv', index=False)

df_merged[['jcode']].to_csv('data/2000/jcode.csv')
df_merged[['jcode','w_j']].to_csv('data/2000/w_j.csv', index=False)
df_merged[['jcode','p_i']].to_csv('data/2000/p_i.csv', index=False)
df_merged[['jcode','q_i']].to_csv('data/2000/q_i.csv', index=False)
df_merged[['jcode','Q_j']].to_csv('data/2000/Q_j.csv', index=False)
df_merged[['jcode','theta_i']].to_csv('data/2000/theta_i.csv', index=False)

df_Pi_ij.to_csv('data/2000/Pi_ij.csv')
df_n_ij.to_csv('data/2000/n_ij.csv')
df_time_ij.to_csv('data/2000/t_ij.csv')
df_cost_ij.to_csv('data/2000/tau_ij.csv')

# 可視化

In [81]:
# 通勤人数の集計
df_commute_ij_agg = pd.DataFrame({
    'jcode': df_commute_ij.index,
    'residents': df_commute_ij.sum(axis=1, numeric_only=True).to_numpy(),
    'workers': df_commute_ij.sum(axis=0, numeric_only=True).to_numpy()
})

# 子供の数の集計
df_children_ij_agg = pd.DataFrame({
    'jcode': df_children_ij.index,
    'children_i': df_children_ij.sum(axis=1, numeric_only=True).to_numpy(),
    'children_j': df_children_ij.sum(axis=0, numeric_only=True).to_numpy(),
})

# 子供の数の平均
df_n_ij_agg = pd.DataFrame({
    'jcode': df_n_ij.index,
    'n_i_ave': df_n_ij[df_n_ij != 0].mean(axis=1, numeric_only=True).to_numpy(),
    'n_j_ave': df_n_ij[df_n_ij != 0].mean(axis=0, numeric_only=True).to_numpy(),
})

df_merged = df_merged.merge(df_commute_ij_agg, left_on='jcode_to', right_on='jcode', how='left').drop(columns=['jcode'])
df_merged = df_merged.merge(df_children_ij_agg, left_on='jcode_to', right_on='jcode', how='left').drop(columns=['jcode'])
df_merged = df_merged.merge(df_n_ij_agg, left_on='jcode_to', right_on='jcode', how='left').drop(columns=['jcode'])

In [82]:
gdf = gpd.read_file('data/setup_2000_2020/jcode.geojson')
gdf['jcode'] = gdf['jcode'].astype(str)
gdf = gdf.merge(df_merged, left_on='jcode', right_on='jcode_to', how='left').drop(columns=['jcode_to'])

In [74]:
from libs_QSM.Plot_gdf import plot_gdf

In [ ]:
plot_gdf(gdf, '2000', 'w_j', '[万円]', lambda x, pos: int(x*300))

In [ ]:
plot_gdf(gdf, '2000', 'q_i', '[万円]', lambda x, pos: int(x*30))

In [ ]:
plot_gdf(gdf, '2000', 'Q_j', '[万円]', lambda x, pos: int(x*30))

In [ ]:
plot_gdf(gdf, '2000', 'residents', '[万人]', lambda x, pos: int(x/10000))

In [ ]:
plot_gdf(gdf, '2000', 'workers', '[万人]', lambda x, pos: int(x/10000))

In [ ]:
gdf['children_j'].std()

In [ ]:
plot_gdf(gdf, '2000', 'children_i', '[万人]', lambda x, pos: int(x/10000))

In [ ]:
plot_gdf(gdf, '2000', 'children_j', '[万人]', lambda x, pos: int(x/10000))

In [ ]:
plot_gdf(gdf, '2000', 'n_i_ave', '[人]', lambda x, pos: x.round(2))

In [ ]:
plot_gdf(gdf, '2000', 'n_j_ave', '[人]', lambda x, pos: x.round(2))

In [ ]:
plot_gdf(gdf, '2000', 'theta_i', '[%]', lambda x, pos: int(x*100))